In [2]:
! pip install transformers datasets evaluate
! pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

In [3]:
###IMPORTS###
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DistilBertTokenizer
import numpy as np
import evaluate

In [4]:
dataset = load_dataset("dariusbotusanu/ro_financial_phrasebank_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3168 [00:00<?, ? examples/s]

Generating test_all_agree split:   0%|          | 0/680 [00:00<?, ? examples/s]

Generating test_75_agree split:   0%|          | 0/680 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3168
    })
    test_all_agree: Dataset({
        features: ['text', 'label'],
        num_rows: 680
    })
    test_75_agree: Dataset({
        features: ['text', 'label'],
        num_rows: 680
    })
})

In [6]:
tokenizer = DistilBertTokenizer.from_pretrained("racai/distilbert-base-romanian-cased", padding='max_length', max_length=45, truncation='max_length')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=45)

# Tokenize the datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

#tokenized_train = dataset['train'].map(tokenization, batched=True)
#tokenized_eval = dataset['test_all_agree'].map(tokenization, batched=True)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/397k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Map:   0%|          | 0/3168 [00:00<?, ? examples/s]

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

In [7]:
tokenized_train = tokenized_datasets['train']
tokenized_eval = tokenized_datasets['test_all_agree']

In [8]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 3168
})

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("racai/distilbert-base-romanian-cased", num_labels=3)

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at racai/distilbert-base-romanian-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(output_dir="/content/drive/MyDrive/RoFinBert/test_trainer", num_train_epochs=30)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()

Step,Training Loss
500,0.333400
1000,0.074300
1500,0.021200
2000,0.007400
2500,0.005600
3000,0.006800
3500,0.004600
4000,0.006100
4500,0.016800
5000,0.001700


TrainOutput(global_step=11880, training_loss=0.02124353330385415, metrics={'train_runtime': 986.2944, 'train_samples_per_second': 96.361, 'train_steps_per_second': 12.045, 'total_flos': 1106536472553600.0, 'train_loss': 0.02124353330385415, 'epoch': 30.0})

In [14]:
trainer.save_model("/content/drive/MyDrive/RoFinBert/model_directory/fine_tuned_model")